In [19]:
import pandas as pd
import numpy as np
import math 

inputs wind and direction, outputs adjusted azimuth & distance<br>
given impact point of wind, calculate implied wind distance & azimuth<br>
calculate length of third side given TSG angle and lengths of the two other sides (TSG = Target, Spotter, Gunner)

In [20]:

def findTSGAngle(spotterToTargetAzimuth, spotterToGunAzimuth):
    azimuthArray = [spotterToTargetAzimuth, spotterToGunAzimuth]
    aziMin = min(azimuthArray)
    aziMax = max(azimuthArray)    


    if (360 - aziMax - aziMin) > 0:
        return min(aziMax - aziMin, 360 - aziMax - aziMin)
    else:
        return (aziMax - aziMin)
    

# print(findTSGAngle(0,90))
# print(findTSGAngle(264,171))


In [21]:
def findDistanceGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance):
    dST = spotterToTargetDistance
    dSG = spotterToGunDistance
    aTSG = findTSGAngle(spotterToTargetAzimuth,spotterToGunAzimuth)

    # Law of Cosines to solve 3rd unknown angle
    # a^2 = b^2 + c^2 − 2*b*c*cos(A)

    # math.radians() inputs degrees, outputs radians
    # math.cos() inputs radians, outputs radians
    # math.degrees inputs radians, outputs degrees


    distGunToTarget = math.sqrt(dST**2 + dSG**2 - 2*dST*dSG*math.cos(math.radians(aTSG)))

    return distGunToTarget

# print(findDistanceGunToTarget(253, 66, 173, 16))

In [22]:
def findTGSAngle(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance):
    # https://courses.lumenlearning.com/suny-osalgebratrig/chapter/non-right-triangles-law-of-cosines/ intuition
    # reference figure 3
    # distance GT = a
    # distance ST = b
    # distance SG = c
    # angle TSG = alpha
    # objective: find angle Beta
    # use equation cos(beta) = (a**2 + c**2 - b**2)/(2*a*c)

    dGT = findDistanceGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)
    dST = spotterToTargetDistance
    dSG = spotterToGunDistance

    # math.acos() inputs radians & outputs radians
    aTGS = math.degrees(math.acos((dGT**2 + dSG**2 - dST**2)/(2*dGT*dSG)))
    return aTGS

# print(findTGSAngle(0,3,90,4))


In [23]:
def findAzimuthGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance):
    aTGS = findTGSAngle(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)
    aTSG = findTSGAngle(spotterToTargetAzimuth, spotterToGunAzimuth)
    aSTG = 180 - (aTGS + aTSG)
    aziSG = spotterToGunAzimuth
    aziST = spotterToTargetAzimuth


    # Create imaginary box around 3 points (Spotter-Target-Gun) box & try to use alternate interior angle theorem

    # Graph 1
    # if aziSG > 180, take aziSG - 180
    # SG > ST
    # ST < 180
    # SG > 180
    # (aziSG - 180) + aTGS

    # Graph 2
    # if aziSG < 180, take aziSG + 180
    # SG > ST
    # ST < 180
    # SG < 180
    # (aziSG + 180) + aTGS

    # Graph 3
    # SG < ST
    # ST > 180
    # SG > 180
    # (aziSG - 180) - aTGS

    # Graph 4
    # SG < ST
    # ST < 180
    # SG < 180
    # same formula as graph 2
    # (aziSG + 180) - aTGS

    # if SG > 180, first term is subtraction
    # if SG < 180, first term is addition

    # if SG > ST, second term is addition
    # if SG < ST, second term is subtraction
    if (aziSG > aziST) and (aziSG > 180):
        return (aziSG - 180) + aTGS

    elif (aziSG > aziST) and (aziSG < 180):
        return (aziSG + 180) + aTGS
    
    elif (aziSG < aziST) and (aziSG > 180):
        return (aziSG - 180) - aTGS

    elif (aziSG < aziST) and (aziSG < 180):
        return (aziSG + 180) - aTGS
        
print(findAzimuthGunToTarget(253, 66, 173, 16))

266.99488180869963


In [24]:
def comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce):
    unadjustedGunToTargetDistance = findDistanceGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)
    unadjustedGunToTargetAzimuth = findAzimuthGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)

    # for when I figure out model
    # return findWindAdjustedGunToTargetAziDist(unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce)
    return [unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance]

# print(findDistanceGunToTarget(254, 65, 173, 17))
# print(findAzimuthGunToTarget(254, 65, 173, 17)) ************ test here

In [25]:
# The plan
# Purpose: build model to predict gun azi/dist given gunAzimuth, gunDistance, windAzimuth, & windForce (multi-variate?)
# Data to work with: spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, spotterToImpactAzimuth, spotterToImpactDistance, windAzimuth, windForce

# Step 0: get data from training island by firing artillery shills, adapt artilleryContinuous in order to generate data (perhaps read & write from excel file function?)

# Step 1: make model to predict impact azimuth and impact distance
# implementation of step 1:
    # model 1: gunToImpactAzimuth:
        # dependent variables: gunToTargetDistance, windAzimuth, windForce
        # independent variable: gunToImpactAzimuth

    # model 2: gunToImpactDistance:
        # dependent variables: gunToTargetDistance, windAzimuth, windForce
        # independent variable: gunToImpactDistance


# Step 2: use goal seek to find necessary gunToTargetAzimuth and gunToTargetDistance to make impact azimuth and impact distance equal to target azimuth and target distance, respectively
# implemention of step 2:
    # model to calculate newGunToTargetAzimuth to hit target
    # Input:
    # Output: the necessary azimuth in order to get impact = target

    # model to calculate newGunToTargetDistance to hit target
    # Input: 
    # Output: the necessary distance in order to get impact = target

# model
    # indepentdent variable =  distance from gun to target
    # dependent variable = difference between predicted impact and actual impact
    # use to find how error changes over distance fired
    # use to find given error at certain distance
    # error that cannot be explained by wind

In [26]:
# https://openpyxl.readthedocs.io/en/stable/

from os.path import exists
from openpyxl import Workbook
import pandas as pd

def getDataForArtilleryModels(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, excelFileName, caliberOfWeapon):
    firingHistoryArray = pd.DataFrame(columns=list(["spotterToTargetAzimuth", "spotterToTargetDistance", "spotterToGunAzimuth", "spotterToGunDistance", 
        "windAzimuth", "windForce", "unadjustedGunToTargetAzimuth", "unadjustedGunToTargetDistance", "gunToImpactAzimuth", "gunToImpactDistance"]))
    print("Keep in mind multiple input responses should be space separated.")

    while True:
        print(firingHistoryArray.tail(2))
        print("\n\n")
        userInput = input("Enter <S> to stop, <C> to change variables, or press <Esc> key to continue. \t").lower() # potentially add another option to change values in dataframe if a mistake was entered

        # saves current data in dataframe to excel file & closes program
        if userInput == "s":
            # if .xlsx file already exists
            if (exists(f"{excelFileName}.xlsx")):
                # add firingHistoryArray dataframe to excel file to save data
                # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html   pd.ExcelWriter section
                # appends firingHistoryArray to existing excel file named whatever the <excelFilName> was, MUST BE IN SAME DIRECTORY AS PROGRAM

                # import excel file, converts to pandas dataframe
                xlToPandas = pd.ExcelFile(f"{excelFileName}.xlsx")
                dataFrameFromPastExcel = xlToPandas.parse(f"{caliberOfWeapon}")
                dataFrameFromPastExcel.reset_index(drop=True, inplace=True)

                # makes the index of the two merge and go from 0 to N, vs restarting from zero
                dataFrameFromPastExcel.reset_index(drop=True, inplace=True)
                firingHistoryArray.reset_index(drop=True, inplace=True)

                # merge historical excel data with new data to upload to Excel file
                combinedDataframe = pd.concat([dataFrameFromPastExcel, firingHistoryArray], ignore_index=True)
                combinedDataframe.drop(combinedDataframe.columns[combinedDataframe.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
                with pd.ExcelWriter(f"{excelFileName}.xlsx", mode = "a", engine="openpyxl", if_sheet_exists="replace") as writer:
                    combinedDataframe.to_excel(writer, sheet_name = f"{caliberOfWeapon}", header="None")
            else:
                # create new .xlsx file and save firingHistoryArray dataframe to it and save file if doesn't already exist
                # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html
                firingHistoryArray.to_excel(f"{excelFileName}.xlsx", sheet_name=caliberOfWeapon)
            print("Exiting..")
            break



        # changes inputs if inputs change, ask user if these are the correct inputs before applying
        elif (userInput == "c"):
            # display possible inputs
            # changes inputs to what the other puts in
            while True:
                currentVariableValuesArray = [spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce]
                print(currentVariableValuesArray)
                variableNumberAndValueToChange = int(input("Enter number 1 through 6 to change the corresponding variable between: spotterToTargetAzimuth(1), spotterToTargetDistance(2), spotterToGunAzimuth(3), spotterToGunDistance(4), windAzimuth(5), windForce(6) \n," + 
                    "and the value to change it to with a space between. Press <Esc> to leave. \t").split())

                if variableNumberAndValueToChange == "":
                    break

                # changes variable to input
                currentVariableValuesArray[int(variableNumberAndValueToChange[0])] = int(variableNumberAndValueToChange[1])

                # assigns values in array to variables 
                spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce = currentVariableValuesArray.split()

                # confirm these are the correct variables?
                print(currentVariableValuesArray)
                userInputToLeaveChange = input("Are these the correct variables? Enter <Y> to leave. \t").lower()
                if userInputToLeaveChange == "y":
                    break

        # once impact hits, ask for impact azimuth and impact distance
        while True:
            impactAzi, impactDist = input("Please enter impact azimuth & impact distance, i.e. spotterToImpactAzimuth & spotterToImpactDistance \t").split()
            print(f"Impact Azimuth: {impactAzi} \n Impact Distance: {impactDist}")
            areImpactCorrectInput = input("Are these impact values correct? Enter <Y> to leave loop.").lower()
            if areImpactCorrectInput == "y":
                break

        impactValuesSpotterToImpact = [int(impactAzi), int(impactDist)]
        impactValuesGunToImpact = [findAzimuthGunToTarget(impactValuesSpotterToImpact[0], impactValuesSpotterToImpact[1], spotterToGunAzimuth, spotterToGunDistance), findDistanceGunToTarget(impactValuesSpotterToImpact[0], impactValuesSpotterToImpact[1], spotterToGunAzimuth, spotterToGunDistance)]
        
        # for when I have a model to calculate adjustments for wind
        # gunToTargetNewVals = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
        # firingHistoryArray = firingHistoryArray.append([spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce] + gunToTargetNewVals + impactValuesGunToImpact)

        gunToTargetNewVals = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
        valuesToStoreInDataframe = [spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce] + gunToTargetNewVals + impactValuesGunToImpact
        firingHistoryArrayLength = len(firingHistoryArray)
        firingHistoryArray.reset_index(drop=True, inplace=True)
        firingHistoryArray.loc[firingHistoryArrayLength] = valuesToStoreInDataframe


# input variables
# (spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, excelFileName, caliberOfWeapon)
# NOTE: cannot have existing Excel file open when saving to it
getDataForArtilleryModels(254,65,173,17,124,4,"artilleryData","120mm")


Keep in mind multiple input responses should be space separated.
Empty DataFrame
Columns: [spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, gunToImpactAzimuth, gunToImpactDistance]
Index: []



Impact Azimuth: 50 
 Impact Distance: 50
   spotterToTargetAzimuth  spotterToTargetDistance  spotterToGunAzimuth  \
0                   254.0                     65.0                173.0   

   spotterToGunDistance  windAzimuth  windForce  unadjustedGunToTargetAzimuth  \
0                  17.0        124.0        4.0                    269.074221   

   unadjustedGunToTargetDistance  gunToImpactAzimuth  gunToImpactDistance  
0                      64.562217          396.472025            60.949868  



Impact Azimuth: 50000 
 Impact Distance: 500000
   spotterToTargetAzimuth  spotterToTargetDistance  spotterToGunAzimuth  \
0                   254.0                    

In [42]:
# https://www.w3schools.com/python/python_ml_multiple_regression.asp
# multilinear regression model
# caliberOfweapon is worksheet name
# model 1: gunToImpactAzimuth:
    # dependent variables: gunToTargetDistance, windAzimuth, windForce
    # independent variable: gunToImpactAzimuth

# model 2: gunToImpactDistance:
    # dependent variables: gunToTargetDistance, windAzimuth, windForce
    # independent variable: gunToImpactDistance
    
from sklearn import linear_model

def findMultilinearRegression(excelFileName, caliberOfWeapon, indVars, depVar):
    xlToPandas = pd.ExcelFile(f"{excelFileName}.xlsx")
    df = xlToPandas.parse(f"{caliberOfWeapon}")
    df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    independentVars = df[indVars]
    dependentVar = df[depVar]
    regr = linear_model.LinearRegression()
    regr.fit(independentVars, dependentVar)
    return regr


gunToImpactAzimuthRegression = findMultilinearRegression("artilleryData", "120mm",["unadjustedGunToTargetAzimuth", "unadjustedGunToTargetDistance","windAzimuth","windForce"],"gunToImpactAzimuth")
gunToImpactDistanceRegression = findMultilinearRegression("artilleryData", "120mm",["unadjustedGunToTargetDistance","windAzimuth","windForce"],"gunToImpactDistance")


In [44]:
predictedgGunToImpactAzimuth = gunToImpactAzimuthRegression.predict([[300, 200,300,4]])
predictedGunToImpactDistance = gunToImpactDistanceRegression.predict([[200,300,4]])
print(predictedgGunToImpactAzimuth[0])
print(predictedGunToImpactDistance[0])

390.5943688599045


C:\Users\matth\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
